## Graph class
- Here is the Graph class you'll be working with. Please take a moment to familiarize yourself with it and its methods. Feel free to ask an LLM to help explain how the class functions!

In [1]:
import random
import heapq
import os
import numpy as np
import itertools

In [2]:
class Graph:
    def __init__(self, directed=False):
        """
        Initialize the Graph.

        Parameters:
        - directed (bool): Specifies whether the graph is directed. Default is False (undirected).

        Attributes:
        - graph (dict): A dictionary to store vertices and their adjacent vertices (with weights).
        - directed (bool): Indicates whether the graph is directed.
        """
        self.graph = {}
        self.directed = directed
    
    def add_vertex(self, vertex):
        """
        Add a vertex to the graph.

        Parameters:
        - vertex: The vertex to add. It must be hashable.

        Ensures that each vertex is represented in the graph dictionary as a key with an empty dictionary as its value.
        """
        if not isinstance(vertex, (int, str, tuple)):
            raise ValueError("Vertex must be a hashable type.")
        if vertex not in self.graph:
            self.graph[vertex] = {}
    
    def add_edge(self, src, dest, weight):
        """
        Add a weighted edge from src to dest. If the graph is undirected, also add from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        - weight: The weight of the edge.
        
        Prevents adding duplicate edges and ensures both vertices exist.
        """
        if src not in self.graph or dest not in self.graph:
            raise KeyError("Both vertices must exist in the graph.")
        if dest not in self.graph[src]:  # Check to prevent duplicate edges
            self.graph[src][dest] = weight
        if not self.directed and src not in self.graph[dest]:
            self.graph[dest][src] = weight
    
    def remove_edge(self, src, dest):
        """
        Remove an edge from src to dest. If the graph is undirected, also remove from dest to src.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.
        """
        if src in self.graph and dest in self.graph[src]:
            del self.graph[src][dest]
        if not self.directed:
            if dest in self.graph and src in self.graph[dest]:
                del self.graph[dest][src]
    
    def remove_vertex(self, vertex):
        """
        Remove a vertex and all edges connected to it.

        Parameters:
        - vertex: The vertex to be removed.
        """
        if vertex in self.graph:
            # Remove any edges from other vertices to this one
            for adj in list(self.graph):
                if vertex in self.graph[adj]:
                    del self.graph[adj][vertex]
            # Remove the vertex entry itself
            del self.graph[vertex]
    
    def get_adjacent_vertices(self, vertex):
        """
        Get a list of vertices adjacent to the specified vertex.

        Parameters:
        - vertex: The vertex whose neighbors are to be retrieved.

        Returns:
        - List of adjacent vertices. Returns an empty list if vertex is not found.
        """
        return list(self.graph.get(vertex, {}).keys())

    def _get_edge_weight(self, src, dest):
        """
        Get the weight of the edge from src to dest.

        Parameters:
        - src: The source vertex.
        - dest: The destination vertex.

        Returns:
        - The weight of the edge. If the edge does not exist, returns infinity.
        """
        return self.graph[src].get(dest, float('inf'))
    
    def __str__(self):
        """
        Provide a string representation of the graph's adjacency list for easy printing and debugging.

        Returns:
        - A string representation of the graph dictionary.
        """
        return str(self.graph)
    

class Graph_Advanced(Graph):
    
    def shortest_path(self, start, end): 
        """
        Calculate the shortest path from a starting node to an ending node in a sparse graph
        with potentially 10000s of nodes. Must run under 0.5 second and find the shortest distance between two nodes.
        
        Parameters:
        start: The starting node.
        end: The ending node.
        
        Returns:
        A tuple containing the total distance of the shortest path and a list of nodes representing that path.
        """

        # Mahtab's code start

        # Priority queue to store (distance, vertex) tuples
        queue = [(0, start)]
        # Dictionary to store the shortest distance to each vertex
        distances = {vertex: float('inf') for vertex in self.graph}
        distances[start] = 0
        # Dictionary to store the path to each vertex
        previous_vertices = {vertex: None for vertex in self.graph}
        
        while queue:
            current_distance, current_vertex = heapq.heappop(queue)
            
            # If we reached the end vertex, we can reconstruct the path
            if current_vertex == end:
                path = []
                while previous_vertices[current_vertex] is not None:
                    path.insert(0, current_vertex)
                    current_vertex = previous_vertices[current_vertex]
                path.insert(0, start)
                return current_distance, path
            
            # If a shorter path to current_vertex has been found, skip this one
            if current_distance > distances[current_vertex]:
                continue
            
            # Explore neighbors
            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight
                
                # Only consider this new path if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_vertices[neighbor] = current_vertex
                    heapq.heappush(queue, (distance, neighbor))
        
        # If there's no path from start to end
        return float('inf'), []
    
        # Mahtab's code end
        
        
        # Your code here
        # return dist, path
    
    def tsp_small_graph(self, start_vertex): 
        """
        Solve the Travelling Salesman Problem for a small (~10 node) complete graph starting from a specified node.
        Required to find the optimal tour. Expect graphs with at most 10 nodes. Must run under 0.5 seconds.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """

        # Mahtab's code start

        # Number of vertices
        n = len(self.graph)
        
        # Initialize memoization table
        memo = {}
        
        # Generate all subsets of vertices
        def tsp_dp(current, visited):
            # If all vertices are visited, return the distance to start vertex
            if visited == (1 << n) - 1:
                return self._get_edge_weight(current, start_vertex)
            
            # If this subproblem has already been solved, return the stored result
            if (current, visited) in memo:
                return memo[(current, visited)]
            
            # Initialize minimum distance
            min_dist = float('inf')
            
            # Try visiting all unvisited vertices
            for vertex in range(n):
                if visited & (1 << vertex) == 0:  # If vertex is not visited
                    dist = self._get_edge_weight(current, vertex) + tsp_dp(vertex, visited | (1 << vertex))
                    min_dist = min(min_dist, dist)
            
            # Store the result in the memoization table
            memo[(current, visited)] = min_dist
            return min_dist
        
        # Reconstruct the path from the memoization table
        def reconstruct_path():
            path = [start_vertex]
            visited = 1 << start_vertex
            current = start_vertex
            
            for _ in range(n - 1):
                next_vertex = min(
                    (vertex for vertex in range(n) if visited & (1 << vertex) == 0),
                    key=lambda vertex: self._get_edge_weight(current, vertex) + memo.get((vertex, visited | (1 << vertex)), float('inf'))
                )
                path.append(next_vertex)
                visited |= (1 << next_vertex)
                current = next_vertex
            
            path.append(start_vertex)
            return path
        
        # Solve the TSP starting from the start_vertex
        total_distance = tsp_dp(start_vertex, 1 << start_vertex)
        path = reconstruct_path()
        
        return total_distance, path
    
        # Mahtab's code end
    
        # Your code here
        # return dist, path
    
    
    def tsp_large_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a large (~1000 node) complete graph starting from a specified node.
        No requirement to find the optimal tour. Must run under 0.5 second and its solution must no
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """
        
        # Mahtab's code start
        
        # Initialize unvisited vertices
        unvisited = set(self.graph.keys())
        unvisited.remove(start)
        
        # Initialize path and distance
        path = [start]
        total_distance = 0
        current_vertex = start
        
        # Use the Nearest Neighbor heuristic
        while unvisited:
            # Find the nearest unvisited vertex
            next_vertex = min(unvisited, key=lambda vertex: self._get_edge_weight(current_vertex, vertex))
            total_distance += self._get_edge_weight(current_vertex, next_vertex)
            path.append(next_vertex)
            current_vertex = next_vertex
            unvisited.remove(next_vertex)
        
        # Return to the starting vertex
        total_distance += self._get_edge_weight(current_vertex, start)
        path.append(start)
        
        return total_distance, path

        # Mahtab's code end

        # Your code here
        # return dist, path

    
    def tsp_medium_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a medium (~300 node) complete graph starting from a specified node.
        Expected to perform better than tsp_large_graph. Must run under 1 second.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """

        # Mahtab's code start

        
        # Mahtab's code end
        
        # Your code here
        # return dist, path
    

## Trial functions here

In [ ]:
class Graph_Advanced(Graph):
    # ... (other methods)

    def tsp_medium_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a medium (~300 node) complete graph starting from a specified node.
        Expected to perform better than tsp_large_graph. Must run under 1 second.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """

        def calculate_total_distance(path):
            return sum(self._get_edge_weight(path[i], path[i + 1]) for i in range(len(path) - 1))

        # Use Nearest Neighbor to get an initial solution
        unvisited = set(self.graph.keys())
        unvisited.remove(start)
        path = [start]
        current_vertex = start
        while unvisited:
            next_vertex = min(unvisited, key=lambda vertex: self._get_edge_weight(current_vertex, vertex))
            path.append(next_vertex)
            current_vertex = next_vertex
            unvisited.remove(next_vertex)
        path.append(start)  # Return to the starting vertex

        total_distance = calculate_total_distance(path)
        return total_distance, path

In [3]:
# # Can you explain me how the above Graph class functions?
# Summary
# The Graph class allows you to create and manipulate a graph data structure.
# You can add vertices and edges, remove them, and query the graph for adjacent vertices and edge weights.
# The class supports both directed and undirected graphs, controlled by the directed attribute.

In [4]:
# You are required to modify a new class, which inherits from the Graph class, by filling in the blank methods listed below. It's entirely acceptable, and often necessary, to add auxiliary (helper) methods to address each problem effectively.

# However, the primary function tasked with solving each problem must retain the names provided below and return the output in the specified order: first the distance, followed by the path. Variable names in the return call are just examples. You can change them, as long as the first argument is the distance and the second is the path, given by a list of node numbers.

## Generate and Test Graph

In [8]:
def generate_graph(nodes, edges=None, complete=False, weight_bounds=(1,600), seed=None):
    """
    Generates a graph with specified parameters, allowing for both complete and incomplete graphs.
    
    This function creates a graph with a specified number of nodes and edges, with options for creating a complete graph, and for specifying the weight bounds of the edges. It uses the Graph_Advanced class to create and manipulate the graph.

    Parameters:
    - nodes (int): The number of nodes in the graph. Must be a positive integer.
    - edges (int, optional): The number of edges to add for each node in the graph. This parameter is ignored if `complete` is set to True. Defaults to None.
    - complete (bool, optional): If set to True, generates a complete graph where every pair of distinct vertices is connected by a unique edge. Defaults to False.
    - weight_bounds (tuple, optional): A tuple specifying the lower and upper bounds (inclusive) for the random weights of the edges. Defaults to (1, 600).
    - seed (int, optional): A seed for the random number generator to ensure reproducibility. Defaults to None.

    Raises:
    - ValueError: If `edges` is not None and `complete` is set to True, since a complete graph does not require specifying the number of edges.

    Returns:
    - Graph_Advanced: An instance of the Graph_Advanced class representing the generated graph, with vertices labeled as integers starting from 0.

    Examples:
    - Generating a complete graph with 5 nodes:
        generate_graph(5, complete=True)
    
    - Generating an incomplete graph with 5 nodes and 2 edges per node:
        generate_graph(5, edges=2)
    
    Note:
    - The function assumes the existence of a Graph_Advanced class with methods for adding vertices (`add_vertex`) and edges (`add_edge`), as well as a method for getting adjacent vertices (`get_adjacent_vertices`).
    """
    random.seed(seed)
    graph = Graph_Advanced()
    if edges is not None and complete:
        raise ValueError("edges must be None if complete is set to True")
    if not complete and edges > nodes:
        raise ValueError("number of edges must be less than number of nodes")
    

    for i in range(nodes):
        graph.add_vertex(i)
    if complete:
        for i in range(nodes):
            for j in range(i+1,nodes):
                weight = random.randint(weight_bounds[0], weight_bounds[1])
                graph.add_edge(i,j,weight)
    else:
        for i in range(nodes):
            for _ in range(edges):
                j = random.randint(0, nodes - 1)
                while (j == i or j in graph.get_adjacent_vertices(i)) and len(graph.get_adjacent_vertices(i)) < nodes - 1:  # Ensure the edge is not a loop or a duplicate
                    j = random.randint(0, nodes - 1)
                weight = random.randint(weight_bounds[0], weight_bounds[1])
                if len(graph.graph[i]) < edges and len(graph.graph[j]) < edges:
                    graph.add_edge(i, j, weight)
    return graph

In [ ]:
%%timeit
g = generate_graph(10, complete=True)
# print(g)

In [ ]:
%%timeit

# g.shortest_path(0, 50)
# 2.16 s ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# g.shortest_path(0, 9000)
# 4.71 s ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## My good functions implementations below

In [ ]:
class Graph_Advanced(Graph):
    
    def shortest_path(self, start, end): 
        """
        Calculate the shortest path from a starting node to an ending node in a sparse graph
        with potentially 10000s of nodes. Must run under 0.5 second and find the shortest distance between two nodes.
        
        Parameters:
        start: The starting node.
        end: The ending node.
        
        Returns:
        A tuple containing the total distance of the shortest path and a list of nodes representing that path.
        """

        # Mahtab's code start

        # Priority queue to store (distance, vertex) tuples
        queue = [(0, start)]
        # Dictionary to store the shortest distance to each vertex
        distances = {vertex: float('inf') for vertex in self.graph}
        distances[start] = 0
        # Dictionary to store the path to each vertex
        previous_vertices = {vertex: None for vertex in self.graph}
        
        while queue:
            current_distance, current_vertex = heapq.heappop(queue)
            
            # If we reached the end vertex, we can reconstruct the path
            if current_vertex == end:
                path = []
                while previous_vertices[current_vertex] is not None:
                    path.insert(0, current_vertex)
                    current_vertex = previous_vertices[current_vertex]
                path.insert(0, start)
                return current_distance, path
            
            # If a shorter path to current_vertex has been found, skip this one
            if current_distance > distances[current_vertex]:
                continue
            
            # Explore neighbors
            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight
                
                # Only consider this new path if it's better
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_vertices[neighbor] = current_vertex
                    heapq.heappush(queue, (distance, neighbor))
        
        # If there's no path from start to end
        return float('inf'), []
    
        # Mahtab's code end
        
        
        # Your code here
        # return dist, path
    
    def tsp_small_graph(self, start_vertex): 
        """
        Solve the Travelling Salesman Problem for a small (~10 node) complete graph starting from a specified node.
        Required to find the optimal tour. Expect graphs with at most 10 nodes. Must run under 0.5 seconds.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """

        # Mahtab's code start

        # Number of vertices
        n = len(self.graph)
        
        # Initialize memoization table
        memo = {}
        
        # Generate all subsets of vertices
        def tsp_dp(current, visited):
            # If all vertices are visited, return the distance to start vertex
            if visited == (1 << n) - 1:
                return self._get_edge_weight(current, start_vertex)
            
            # If this subproblem has already been solved, return the stored result
            if (current, visited) in memo:
                return memo[(current, visited)]
            
            # Initialize minimum distance
            min_dist = float('inf')
            
            # Try visiting all unvisited vertices
            for vertex in range(n):
                if visited & (1 << vertex) == 0:  # If vertex is not visited
                    dist = self._get_edge_weight(current, vertex) + tsp_dp(vertex, visited | (1 << vertex))
                    min_dist = min(min_dist, dist)
            
            # Store the result in the memoization table
            memo[(current, visited)] = min_dist
            return min_dist
        
        # Reconstruct the path from the memoization table
        def reconstruct_path():
            path = [start_vertex]
            visited = 1 << start_vertex
            current = start_vertex
            
            for _ in range(n - 1):
                next_vertex = min(
                    (vertex for vertex in range(n) if visited & (1 << vertex) == 0),
                    key=lambda vertex: self._get_edge_weight(current, vertex) + memo.get((vertex, visited | (1 << vertex)), float('inf'))
                )
                path.append(next_vertex)
                visited |= (1 << next_vertex)
                current = next_vertex
            
            path.append(start_vertex)
            return path
        
        # Solve the TSP starting from the start_vertex
        total_distance = tsp_dp(start_vertex, 1 << start_vertex)
        path = reconstruct_path()
        
        return total_distance, path
    
        # Mahtab's code end
    
        # Your code here
        # return dist, path
    
    
    def tsp_large_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a large (~1000 node) complete graph starting from a specified node.
        No requirement to find the optimal tour. Must run under 0.5 second and its solution must no
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """
        
        # Mahtab's code start
        
        # Initialize unvisited vertices
        unvisited = set(self.graph.keys())
        unvisited.remove(start)
        
        # Initialize path and distance
        path = [start]
        total_distance = 0
        current_vertex = start
        
        # Use the Nearest Neighbor heuristic
        while unvisited:
            # Find the nearest unvisited vertex
            next_vertex = min(unvisited, key=lambda vertex: self._get_edge_weight(current_vertex, vertex))
            total_distance += self._get_edge_weight(current_vertex, next_vertex)
            path.append(next_vertex)
            current_vertex = next_vertex
            unvisited.remove(next_vertex)
        
        # Return to the starting vertex
        total_distance += self._get_edge_weight(current_vertex, start)
        path.append(start)
        
        return total_distance, path

        # Mahtab's code end

        # Your code here
        # return dist, path

    
    def tsp_medium_graph(self, start): 
        """
        Solve the Travelling Salesman Problem for a medium (~300 node) complete graph starting from a specified node.
        Expected to perform better than tsp_large_graph. Must run under 1 second.
        
        Parameters:
        start: The starting node.
        
        Returns:
        A tuple containing the total distance of the tour and a list of nodes representing the tour path.
        """

        # Mahtab's code start

        
        # Mahtab's code end
        
        # Your code here
        # return dist, path
    